<h1><center>CMPE 462 - Project 2<br>Implementing an SVM Classifier<br>Due: May 18, 2020, 23:59</center></h1>

* **Student ID1:** 2015400249
* **Student ID2:** 2016400216
* **Student ID3:** 2014400258

## Overview

In this project, you are going to implement SVM. For this purpose, a data set (data.mat) is given to you. You can load the mat dataset into Python using the function `loadmat` in `Scipy.io`. When you load the data, you will obtain a dictionary object, where `X` stores the data matrix and `Y` stores the labels. You can use the first 150 samples for training and the rest for testing. In this project, you will use the software package [`LIBSVM`](http://www.csie.ntu.edu.tw/~cjlin/libsvm/) to implement SVM. Note that `LIBSVM` has a [`Python interface`](https://github.com/cjlin1/libsvm/tree/master/python), so you can call the SVM functions in Python. 

## Task 1 - 30 pts

Train a hard margin linear SVM and report both train and test classification accuracy.

In [30]:
import numpy as np
import pandas as pd
import scipy.io as sio
from libsvm.svmutil import *
from contextlib import contextmanager
import sys, os
from cvxopt import matrix, solvers


@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

def calculateAccuracy(array1, array2):
    correct_count = 0
    size_data = len(array2)
    for i in range(len(array1[0])):
        val1 = array1[0][i]
        val2 = array2[i][0]
        #print(val1, val2 )
        if  val1>0 and val2>0:
            correct_count += 1
                #print("correct")
        elif val1<0 and val2<0:
            correct_count += 1
                #print("correct")
        #else:
            #print("not correct")

    accuracy = correct_count/size_data
    print("Accuracy : ", accuracy)

def countSV(array1, array2):
    sv_count = 0
    for i in range(len(array1[2])):
        val = array1[2][i] * array2[i]
        if val == 1:
            sv_count += 1

    print("sv count : " , sv_count)


data = sio.loadmat("data.mat")
data_x = np.array(data["X"])
data_y = np.array(data["Y"])

train_x = np.array(data_x[0:150,:])
train_y = np.array(data_y[0:150,:])

test_x = np.array(data_x[150:,:])
test_y = np.array(data_y[150:,:])
print(np.shape(test_x))
print(np.shape(test_y))



(120, 13)
(120, 1)


In [31]:
#IMPLEMENT HARD MARGIN SVM

#px = svm_problem(train_y.ravel(), train_x)

print(train_x)
prob = svm_problem(train_y.ravel(), train_x)
m = svm_train(prob, '-t 0 -c 100000') #train with linear kernel and c = 100000

with suppress_stdout():
    train_prediction = svm_predict(train_y,train_x, m)
    
    

[[ 0.708333  1.        1.       ...  0.        1.       -1.      ]
 [ 0.583333 -1.        0.333333 ...  0.       -1.        1.      ]
 [ 0.166667  1.       -0.333333 ... -1.       -1.        1.      ]
 ...
 [-0.541667  1.        1.       ...  0.       -1.        1.      ]
 [ 0.208333  1.        0.333333 ... -1.        0.333333  1.      ]
 [-0.5      -1.        0.333333 ... -1.       -1.       -1.      ]]


In [32]:
calculateAccuracy(train_prediction, train_y)

size_test = len(test_y)
with suppress_stdout():
    test_prediction = svm_predict(test_y,test_x, m)


calculateAccuracy(test_prediction, test_y)


Accuracy :  0.9
Accuracy :  0.825


## Task 2 - 40 pts

Train soft margin SVM for different values of the parameter $C$, and with different kernel functions. Systematically report your results. For instance, report the performances of different kernels for a fixed $C$, then report the performance for different $C$ values for a fixed kernel, and so on.

In [33]:
prob_soft = svm_problem(train_y.ravel(), train_x)
m_soft_kernel1 = svm_train(prob_soft, '-t 0 -c 3') #train with linear kernel and c = 0
m_soft_kernel2 = svm_train(prob_soft, '-t 3 -c 3') #train with linear kernel and c = 0

In [34]:
with suppress_stdout():
    test_prediction_sm_kernel1 = svm_predict(test_y,test_x, m_soft_kernel1)
    test_prediction_sm_kernel2 = svm_predict(test_y,test_x, m_soft_kernel2)
    
calculateAccuracy(test_prediction_sm_kernel1, test_y)
calculateAccuracy(test_prediction_sm_kernel2, test_y)


Accuracy :  0.8416666666666667
Accuracy :  0.825


In [35]:
m_soft_c1 = svm_train(prob_soft, '-t 2 -c 1') #train with linear kernel and c = 0
m_soft_c2 = svm_train(prob_soft, '-t 2 -c 100') #train with linear kernel and c = 0
m_soft_c3 = svm_train(prob_soft, '-t 2 -c 1000') #train with linear kernel and c = 0

In [36]:
with suppress_stdout():
    test_prediction_sm_c1 = svm_predict(test_y,test_x, m_soft_c1)
    test_prediction_sm_c2 = svm_predict(test_y,test_x, m_soft_c2)
    test_prediction_sm_c3 = svm_predict(test_y,test_x, m_soft_c3)

calculateAccuracy(test_prediction_sm_c1, test_y)
calculateAccuracy(test_prediction_sm_c2, test_y)
calculateAccuracy(test_prediction_sm_c3, test_y)

Accuracy :  0.8416666666666667
Accuracy :  0.7833333333333333
Accuracy :  0.7666666666666667


## Task 3 - 15 pts

Please report how the number of support vectors changes as the value of $C$ increases (while all other parameters remain the same). Discuss whether your observations match the theory.

In [37]:
sv_c1 = m_soft_c1.get_SV()
sv_c2 = m_soft_c2.get_SV()
sv_c3 = m_soft_c3.get_SV()
print(np.shape(sv_c1))
print(np.shape(sv_c2))
print(np.shape(sv_c3))

(83,)
(70,)
(64,)


## Task 4 - 15 pts

Please investigate the changes in the hyperplane when you remove one of the support vectors, vs., one data point that is not a support vector.

In [38]:

def findWeightsAndBias(sv, m, sv_coef):
    sv_new = np.zeros((len(sv), 13))
    for i in range(len(sv)):
        temp = np.zeros([13,1])
        for k,v in support_vectors[i].items():
            if k != -1:
                temp[k-1] = v
        sv_new[i] = temp.ravel()
    sv_new = np.transpose(sv_new) #convert sv_new to 13x49 matrix
    w = (sv_new @ sv_coef); 
    rho = m.rho[0]
    b = -rho
    print(w) #weights
    print(b) #bias
    
    
support_vectors = m.get_SV()
sv_coef = np.asarray(m.get_sv_coef()).ravel()
sv_indices = np.asarray(m.get_sv_indices())
#print(sv_indices) #index 3 is index 2 at train_x


findWeightsAndBias(support_vectors, m, sv_coef) # print weights and bias without removing any data point


train_x_without_sv = np.delete(train_x, 2, 0)
train_y_without_sv = np.delete(train_y, 2, 0)
prob1 = svm_problem(train_y_without_sv.ravel(), train_x_without_sv)
m_without_sv = svm_train(prob1, '-t 0 -c 100000') #train with linear kernel and c = 100000
sv1 = m_without_sv.get_SV()
sv_coef1 = np.asarray(m_without_sv.get_sv_coef()).ravel()
findWeightsAndBias(sv1, m_without_sv, sv_coef1) # print weights and bias after removing a sv
    
    
train_x_without_datapoint = np.delete(train_x, 1, 0)
train_y_without_datapoint = np.delete(train_y, 1, 0)
prob2 = svm_problem(train_y_without_datapoint.ravel(), train_x_without_datapoint) 
m_without_nonsv = svm_train(prob2, '-t 0 -c 100000') #train with linear kernel and c = 100000
sv2 = m_without_nonsv.get_SV()
sv_coef2 = np.asarray(m_without_nonsv.get_sv_coef()).ravel()
findWeightsAndBias(sv2, m_without_nonsv, sv_coef2) # print weights and bias after removing non sv data point





[-1.13365625  0.56672624  1.10960429  0.85136329  0.92456137  0.1672546
  0.23397229 -1.12376604 -0.02407569 -0.29319709  0.36764449  2.81462132
  0.46803858]
1.8638659415742513
[  50499.20498987  532426.00875612   43761.04746219  -16365.17700442
 -206856.98829529 -128091.66283821   24252.98596245 -220787.19225765
  162850.38462649  198797.64097959  -79351.50416807 -107925.96496495
  475223.71999137]
1.701769970510478
[ 107176.47024318   -1075.30318506  -13286.71191302  -24662.76586264
   98475.27858452  -26401.6769624  -235249.86830264   44492.57364472
 -292805.35398981  -61798.22284671  -51815.63094628  126210.73220312
  199539.09501501]
2.379365035302916


### Bonus Task - 10 pts

Use Python and [CVXOPT](http://cvxopt.org) QP solver to implement the hard margin SVM. 

In [1]:
from cvxopt import matrix, solvers

Q = matrix([[0.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, 0.0, 1.0]])
p = matrix([0.0, 0.0, 0.0])
A = matrix([[-1.0, -1.0, 1.0, 1.0], [0.0, -2.0, 2.0, 3.0], [0.0, -2.0, 0.0, 0.0]])
c = matrix([-1.0, -1.0, -1.0, -1.0])
sol=solvers.qp(Q,p,A,c)

print(sol['x'])

     pcost       dcost       gap    pres   dres
 0:  3.2653e-01  1.9592e+00  6e+00  2e+00  4e+00
 1:  1.5796e+00  8.5663e-01  7e-01  0e+00  2e-15
 2:  1.0195e+00  9.9227e-01  3e-02  1e-16  1e-15
 3:  1.0002e+00  9.9992e-01  3e-04  2e-16  2e-15
 4:  1.0000e+00  1.0000e+00  3e-06  3e-16  7e-16
 5:  1.0000e+00  1.0000e+00  3e-08  4e-16  7e-16
Optimal solution found.
[ 1.00e+00]
[-1.00e+00]
[ 1.00e+00]

